
# RBK Football2025 — YOLO Setup (Merged Multi-Match)

This notebook is tailored for the dataset structure you described:

```
Football2025/
  RBK-AALESUND/
    aalesund_cvat_overlay.mp4
    aalesund.mp4
    annotations.xml
    data/
      images/train/*.png
    labels/
      train/*.txt
    data.yaml
    train.txt
  RBK-BODO/
  RBK-FREDRIKSTAD/
  RBK-HamKam/
  RBK-VIKING/
```

It will:
1. Auto-discover the `RBK-*` folders under `Football2025`.
2. Collect images from `data/images/train` and matched labels from `labels/train`.
3. Create a **merged YOLO dataset** with `images/train`, `images/val`, `labels/train`, `labels/val` using **symlinks** (no copying).
4. Write a `data.yaml` pointing to these merged splits.
5. Provide training/eval cells for Ultralytics YOLO.


## 0) Environment check

## 1) Paths & discovery

## 2) Merge per-match folders into a single YOLO dataset

## 3) Write `data.yaml` for Ultralytics

## 4) (Optional) Quick stats

## 5) Train YOLO

## 6) Evaluate (Precision, Recall, mAP@50, mAP@[0.5:0.95])


## Notes
- We used **symlinks** to avoid duplicating data. If your environment disallows symlinks, switch `symlink(...)` to copy files instead.
- We performed a global **90/10 train/val split**. If you prefer per-match splits or using the existing `train.txt` files, we can adapt easily.
- The labels are assumed to already be in **YOLO detection format** with classes `[0=player, 1=referee, 2=ball]`. If not, we can parse `annotations.xml` (CVAT) and convert explicitly.
